### Change the Directory Location

In [1]:
import os
os.chdir('../')
%pwd

'n:\\DataScience\\MyProjects\\DeepLearning-Project-With-MLFlow-DVC-Deployment'

### Update the entity

In [2]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_validation_split: float
    params_interpolation: str
    params_validation_data_generator_subset: str
    params_training_data_generator_subset: str
    params_is_validation_data_generator_shuffle: bool
    params_is_training_data_generator_shuffle: bool
    params_rotation_range: int
    params_is_horizontal_flip: bool
    params_width_shift_range: float 
    params_height_shift_range: float
    params_shear_range: float
    params_zoom_range: float
    source_dir: Path

### Update the configuration manager in src config

In [3]:
from DLProject.constants import *
from DLProject.utils.common import read_yaml, create_directories
import tensorflow as tf

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training = self.config.training
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, training.source_dir)
        create_directories([
            Path(training.root_dir)
        ])

        training_config = TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path=Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_validation_split=params.VALIDATION_SPLIT,
            params_interpolation=params.INTERPOLATION,
            params_validation_data_generator_subset=params.VALIDATION_DATA_GENERATOR_SUBSET,
            params_training_data_generator_subset=params.TRAINING_DATA_GENERATOR_SUBSET,
            params_is_validation_data_generator_shuffle=params.VALIDATION_DATA_GENERATOR_SHUFFLE,
            params_is_training_data_generator_shuffle=params.TRAINING_DATA_GENERATOR_SHUFFLE,
            params_rotation_range=params.ROTATION_RANGE,
            params_is_horizontal_flip=params.HORIZONTAL_FLIP,
            params_width_shift_range=params.WIDTH_SHIFT_RANGE,
            params_height_shift_range=params.HEIGH_SHIFT_RANGE,
            params_shear_range=params.SHEAR_RANGE,
            params_zoom_range=params.ZOOM_RANGE,
            source_dir=Path(training.source_dir)
        )

        return training_config

### Update the components

In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [6]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )

    def train_valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=self.config.params_validation_split
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation=self.config.params_interpolation
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset=self.config.params_validation_data_generator_subset,
            shuffle=self.config.params_is_validation_data_generator_shuffle,
            **dataflow_kwargs
        )

        if self.config.params_is_augmentation:
            train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=self.config.params_rotation_range,
                horizontal_flip=self.config.params_is_horizontal_flip,
                width_shift_range=self.config.params_width_shift_range,
                height_shift_range=self.config.params_height_shift_range,
                shear_range=self.config.params_shear_range,
                zoom_range=self.config.params_zoom_range,
                **datagenerator_kwargs
            )
        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset=self.config.params_training_data_generator_subset,
            shuffle=self.config.params_is_training_data_generator_shuffle,
            **dataflow_kwargs
        )

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

    def train(self):
        self.steps_per_epoch = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_generator
        )

        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

### Update the pipeline

In [7]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train()
    
except Exception as e:
    raise e

[2023-10-11 10:10:06,861: INFO: common: YAML file: config\config.yaml loaded successfully]
[2023-10-11 10:10:07,171: INFO: common: YAML file: params.yaml loaded successfully]
[2023-10-11 10:10:07,184: INFO: common: Created directory at: artifacts]
[2023-10-11 10:10:07,189: INFO: common: Created directory at: artifacts\training]
Found 2487 images belonging to 4 classes.
Found 9959 images belonging to 4 classes.
622/622 [==============================] - 9431s 15s/step - loss: 13.8444 - accuracy: 0.4158 - val_loss: 10.8396 - val_accuracy: 0.6306
